# Q1（WordPiece 算法介绍）：
    WordPiece是一种用于文本分词的算法，最初由Google在BERT模型中提出。其主要目标是解决词汇表外词汇（OOV）的问题，同时提高模型的表现。WordPiece通过将单词分解为更小的子词单位，使得即使是未见过的单词也能通过已有的子词组合来表示。
### 具体实现步骤：
初始化词汇表：
从字符级别开始，词汇表最初只包含所有的单个字符和一些特殊符号（如[UNK]、[CLS]、[SEP]等）。

统计词频：
扫描训练数据，记录每个词的出现频率。

生成子词单元：
在每一轮迭代中，计算所有可能的子词对及其出现频率。
选择频率最高的子词对，将其合并为一个新的子词，并更新词汇表。
重复此过程，直到达到预设的词汇表大小或没有更多可合并的子词对。

分词过程：
对输入文本进行分词时，从左到右尝试匹配最长的子词单位，并逐步构建出完整的词汇表示。


# Q2（使用SentencePiece在西游记数据集上训练分词器，西游记.txt_数据集-阿里云天池 (aliyun.com)，并进行测试）：

In [ ]:
import sentencepiece as spm
import jieba

def train_sentencepiece(input_file, model_prefix, vocab_size):
    spm.SentencePieceTrainer.train(f'--input={input_file} --model_prefix={model_prefix} --vocab_size={vocab_size}')

def load_sentencepiece_model(model_file):
    sp = spm.SentencePieceProcessor(model_file=model_file)
    return sp

def sentencepiece_tokenize(sp, text):
    pieces = sp.encode(text, out_type=str)
    return pieces

def sentencepiece_detokenize(sp, pieces):
    return sp.decode(pieces)

def jieba_tokenize(text):
    return list(jieba.cut(text))

if __name__ == "__main__":
    input_file = '西游记.txt'
    model_prefix = 'xiyouji'
    vocab_size = 10000

    train_sentencepiece(input_file, model_prefix, vocab_size)

    sp = load_sentencepiece_model(f'{model_prefix}.model')

    text = '''石猴笑道：“这股水乃是桥下冲贯石窍，倒挂下
    来遮闭门户的。桥边有花有树，乃是一座石房。房内有石锅石
    灶、石碗石盆、石床石凳，中间一块石碣上，镌着‘花果山福
    地，水帘洞洞天’。真个是我们安身之处。里面且是宽阔，容
    得千百口老小。我们都进去住，也省得受老天之气。'''

    sp_pieces = sentencepiece_tokenize(sp, text)
    print("SentencePiece 分词结果:", sp_pieces)

    # 反分词
    sp_original_text = sentencepiece_detokenize(sp, sp_pieces)
    print("SentencePiece 重构文本:", sp_original_text)

    jieba_segments = jieba_tokenize(text)
    print("jieba 分词结果:", jieba_segments)

## 运行结果：

![Alt](01py.png)

# Q3：请解释embedding层的作用，重点介绍为什么使用positional embedding以及positional embedding的原理
在 Transformer 模型中，Embedding 层的作用是将离散的词汇（例如单词或子词）转换为连续的向量表示，以便神经网络能够处理这些数据。

**词汇嵌入 (Word Embedding)**
作用: 词汇嵌入的主要目的是将离散的词汇映射到一个高维向量空间中，每个词汇都有一个固定维度的向量表示。例如，如果词汇嵌入维度是 512，那么每个词汇将被表示为一个 512 维的向量。
 通过这种方式，语义相似的词汇在向量空间中会接近，从而捕捉词汇之间的语义关系。它还可以减少词汇稀疏性，提高计算效率。

**位置编码 (Positional Embedding)**
自注意力机制的特点：
Transformer 使用的是自注意力机制，它允许网络并行处理输入序列中的所有元素，而不是逐个处理。这带来了效率的提升，但同时也带来了一个问题：模型无法捕捉序列中元素的顺序信息。在自然语言处理中，词汇的顺序对于理解句子意义至关重要。例如，“我爱你”和“你爱我”虽然包含相同的词汇，但顺序不同，含义截然不同。如果模型无法捕捉到顺序信息，就很难正确理解和生成语言。位置编码通过正弦和余弦函数生成，确保对齐的两个位置编码之间具有一定的平滑变化和唯一性。这些编码的设计具有周期性和递增性，允许模型捕捉相对位置。正弦和余弦的组合确保了不同位置的编码是唯一的，并且编码之间的距离随位置增大而指数级变大，这有利于模型感知远距离依赖关系。在实际使用中，位置编码被添加到词汇嵌入上，得到一个既包含语义信息又包含位置信息的向量。这使得每个词汇的表示不仅反映了词汇本身的语义，也包含了它在序列中的位置。


# Q4（选做）: 完成下列positional embedding接口的实现...

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, input_dim, max_len=1000):
        super(PositionalEncoding, self).__init__()
        
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, input_dim, 2).float() * (-math.log(10000.0) / input_dim))
        
        pe = torch.zeros(max_len, input_dim)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        """
        Args:
            x: Tensor of shape (batch_size, seq_len, input_dim)
        Returns:
            Tensor of shape (batch_size, seq_len, input_dim)
        """
        x = x + self.pe[:x.size(1), :]
        return x

# Q5：如何理解注意力机制中的key、value、query的作用，试着从矩阵变化的角度来解释其输入X维度的变化

我的理解是，Query是自己被修饰的各种可能的集合，Key是自己修饰他人的各种可能的集合。 比如creature的Q，和blue的K，两者做点积如果值较大就知道哦blue很可能是creature的修饰词。在注意力机制中，`query`、`key` 和 `value` 是关键概念，理解它们的作用有助于把握模型的工作原理。以下是从矩阵变化的角度对输入 `X` 维度变化的解释：

### 1. 输入矩阵 `X`

假设输入矩阵 `X` 的维度为 `(N, T, D)`，其中：
- `N` 是批量大小（batch size）。
- `T` 是序列长度（例如，词汇数）。
- `D` 是每个输入向量的维度（特征维度）。

### 2. 线性变换

我们通过三个不同的权重矩阵将输入 `X` 投影到 `Q`、`K` 和 `V`：

- **Query (Q)**: 
  \[
  $Q = XW_Q \quad (N, T, D) \times (D, d_k) \rightarrow (N, T, d_k)$
  \]

- **Key (K)**: 
  \[
  $K = XW_K \quad (N, T, D) \times (D, d_k) \rightarrow (N, T, d_k)$
  \]

- **Value (V)**: 
  \[
  $V = XW_V \quad (N, T, D) \times (D, d_v) \rightarrow (N, T, d_v)$
  \]

这里，`d_k` 和 `d_v` 是 `query` 和 `value` 的维度，通常是 `D` 的缩小维度。

### 3. 注意力计算

接下来，计算注意力得分：
\[
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
\]

- **矩阵变化**：
  - \( K^T \) 的维度变为 `(N, d_k, T)`，通过点积计算得分后，得到的结果为 `(N, T, T)`。
  - 通过 softmax 处理后，得分用于加权 `V`，最终输出的维度为 `(N, T, d_v)`。

### 4. 结果

最终，`query` 通过与 `key` 的交互，决定了如何加权 `value`，以获得对每个位置的上下文信息。这一过程使得模型能够动态地聚焦于输入序列中不同的部分，从而增强表示能力。


 # Q6：完成简单版本的多头注意力的实现，接口如下：....

In [ ]:
class SimpleMultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(SimpleMultiHeadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == embed_dim, 

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, query, key, value, mask=None):
        N, T, C = query.shape

        Q = self.q_proj(query).view(N, T, self.num_heads, self.head_dim).transpose(1, 2)  # (N, num_heads, T, head_dim)
        K = self.k_proj(key).view(N, T, self.num_heads, self.head_dim).transpose(1, 2)    # (N, num_heads, T, head_dim)
        V = self.v_proj(value).view(N, T, self.num_heads, self.head_dim).transpose(1, 2)  # (N, num_heads, T, head_dim)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)  # (N, num_heads, T, T)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attention_weights = F.softmax(scores, dim=-1)  # (N, num_heads, T, T)

        output = torch.matmul(attention_weights, V)  # (N, num_heads, T, head_dim)

        output = output.transpose(1, 2).contiguous().view(N, T, self.embed_dim)  # (N, T, embed_dim)

        output = self.out_proj(output)  # (N, T, embed_dim)
        
        return output, attention_weights

# Q7：请解释这样做的好处(在llama系列中对注意力机制做了相应的变化...)

1. 提高并行计算效率
通过将 query 头分组，每组 query 头可以与对应的 key 和 value 并行计算，这样可以提高计算效率和速度。同时，这也可以更好地利用硬件资源，如 GPU 和 TPU 的并行计算能力。

2. 减少计算开销
因为每组 query 头仅与对应的 key 和 value 交互，而不是所有头都共享一组 key 和 value，计算量会相应减少，特别是在处理长序列输入时，效率提升明显。

3. 细粒度的上下文捕获
细化到组内部的注意力机制提供了一种细粒度的方式来捕获不同上下文之间的关系。通过分组，不同的 query 头组可以专注于序列中不同的方面或特征，从而细致地捕获并表示上下文信息。

4. 增强模型的多样性
分组的 query 头与不同的 key 和 value 组合可以增加模型的多样性。不同组的 query 头可以捕获到不同层次和不同类型的关系信息，从而提升模型的通用性和表现力。

5. 更灵活的注意力机制
通过将 query 头分组，可以更灵活地调整每组注意力机制的配置。例如，不同的组可以选择不同层次的语义或不同类型的特征，从而增强整体模型的灵活性和适应性。

# Q8：完成GQA的具体实现，接口与SimpleMultiHeadAttention 一致

In [ ]:
class GroupedQueryAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, num_groups):
        super(GroupedQueryAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_groups = num_groups
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == embed_dim, 
        assert num_heads % num_groups == 0, 
        
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, query, key, value, mask=None):
        N, T, _ = query.shape
        group_size = self.num_heads // self.num_groups

        Q = self.q_proj(query).view(N, T, self.num_heads, self.head_dim).transpose(1, 2)  # (N, num_heads, T, head_dim)
        K = self.k_proj(key).view(N, T, self.num_heads, self.head_dim).transpose(1, 2)    # (N, num_heads, T, head_dim)
        V = self.v_proj(value).view(N, T, self.num_heads, self.head_dim).transpose(1, 2)  # (N, num_heads, T, head_dim)

        outputs = []
        attn_weights = []

        for g in range(self.num_groups):
            q_group = Q[:, g * group_size:(g + 1) * group_size, :, :]  # (N, group_size, T, head_dim)
            k_group = K[:, g * group_size:(g + 1) * group_size, :, :]  # (N, group_size, T, head_dim)
            v_group = V[:, g * group_size:(g + 1) * group_size, :, :]  # (N, group_size, T, head_dim)

            scores = torch.matmul(q_group, k_group.transpose(-2, -1)) / math.sqrt(self.head_dim)  # (N, group_size, T, T)
            if mask is not None:
                scores = scores.masked_fill(mask == 0, float('-inf'))
            attn_weights_group = F.softmax(scores, dim=-1)  # (N, group_size, T, T)
            attn_weights.append(attn_weights_group)

            output_group = torch.matmul(attn_weights_group, v_group)  # (N, group_size, T, head_dim)
            outputs.append(output_group)

        output = torch.cat(outputs, dim=1)  # (N, num_heads, T, head_dim)
        output = output.transpose(1, 2).contiguous().view(N, T, self.embed_dim)  # (N, T, embed_dim)
        output = self.out_proj(output)  # (N, T, embed_dim)

        attn_weights = torch.cat(attn_weights, dim=1)  # (N, num_heads, T, T)
        return output, attn_weights

# Q9：完成Feed-Forward层的代码实现：

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(PositionwiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Args:
            x: Tensor of shape (batch_size, seq_len, input_dim)
        
        Returns:
            Tensor of shape (batch_size, seq_len, input_dim)
        """
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Q10： 介绍Layer Normalization的原理，并分析它与batch normalization的区别，以及为什么Transformer 用Layer Normalization而不用batch normalization？

Layer Normalization（层归一化）原理
1. 定义
Layer Normalization（层归一化）在神经网络中作用于每个数据点，而不是像 Batch Normalization（批次归一化）那样作用于整个 mini-batch。具体来说，Layer Normalization 在计算归一化时，对于每个输入样本的特征维度进行归一化。

2. 数学公式
给定输入 x，形状为 (batch_size, seq_len, feature_dim)，对于每个特征维度 i，Layer Normalization 的计算步骤如下：

计算均值：
$   \mu = \frac{1}{H} \sum_{i=1}^{H} x_i
  $
 
 
 

计算方差：
$   \sigma^2 = \frac{1}{H} \sum_{i=1}^{H} (x_i - \mu)^2
  $
 
 
 
 
 

归一化：
$   \hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}}
  $
 
 
 
 
 
 

缩放和平移：
$   y_i = \gamma \hat{x}_i + \beta
  $
 
 
 

其中：

H 是特征维度的大小。
$\epsilon$ 是一个小常数，防止分母为零。
$\gamma$ 和 $\beta$ 是可学习的参数，用于缩放和平移。
与 Batch Normalization 的区别
1. 计算范围
Batch Normalization：在 mini-batch 上计算均值和方差，每个特征在整个 mini-batch 上进行归一化。
$  \hat{x}^{(k)} = \frac{x^{(k)} - \mu_B^{(k)}}{\sqrt{(\sigma_B^{(k)})^2 + \epsilon}}
 $
 
 
 
 
 
 
 
 

其中，k 表示特征维度，B 表示 mini-batch 的大小。

Layer Normalization：在每个单独的样本的特征维度上计算均值和方差，即在特征维度 H 上进行归一化。
$  \hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}}
 $
 
 
 
 
 
 

其中，i 表示特征维度。

2. 适用场景
Batch Normalization：由于在 mini-batch 上计算归一化参数，适用于卷积神经网络（CNN）和全连接网络（FCN），能够利用大批量数据的统计特性。
Layer Normalization：独立于 mini-batch，适用于循环神经网络（RNN）和 Transformer 等，需要对每个时间步或输入样本进行归一化的网络。
为什么 Transformer 使用 Layer Normalization 而不是 Batch Normalization
序列建模的独立性：

Transformer 模型中的每个输入序列在处理时独立于其他序列。Batch Normalization 依赖于 mini-batch 的统计信息，而序列间可能不具有一致性，这会引入噪声。
Layer Normalization 在每个单独样本的特征上进行归一化，与其他样本无关，适合序列建模任务。
在线预测和序列处理：

在预测阶段，特别是在线预测时，可能只有一个样本输入，Batch Normalization 无法有效计算 mini-batch 的统计特性。
Layer Normalization 不依赖于 mini-batch，从而在处理单个样本时更加稳定和有效。
全局依赖特性的捕捉：

Transformer 需要捕捉输入序列中全局依赖关系，而不是局部依赖。Layer Normalization 能够更好地保留序列全局信息，而 Batch Normalization 更适合捕捉局部统计特性。



# Q11（选做）：实现Layer Normalization

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, dim, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(dim))
        self.beta = nn.Parameter(torch.zeros(dim))
        self.eps = eps

    def forward(self, x):
        """
        Args:
            x: Tensor of shape (batch_size, seq_len, dim)

        Returns:
            out: Tensor of shape (batch_size, seq_len, dim)
        """
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)

        out = (x - mean) / (std + self.eps)

        out = self.gamma * out + self.beta

        return out


# Q12：现在你应该可以完成整个transformer的搭建工作，给定相应接口完成transformer的构建：

In [ ]:
import torch
from torch import nn
from typing import Optional
from dataclasses import dataclass

@dataclass
class ModelArgs:
    embed_dim: int
    num_heads: int
    ff_hidden_dim: int
    num_layers: int
    input_dim: int
    output_dim: int
    dropout: float = 0.1

class LayerNorm(nn.Module):
    def __init__(self, dim, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(dim))
        self.beta = nn.Parameter(torch.zeros(dim))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        out = (x - mean) / (std + self.eps)
        out = self.gamma * out + self.beta
        return out

class TransformerBlock(nn.Module):
    def __init__(self, layer_id: int, embed_dim: int, num_heads: int, ff_hidden_dim: int, dropout: float):
        super().__init__()
        self.layer_id = layer_id
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.layernorm1 = LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_hidden_dim, embed_dim)
        )
        self.dropout2 = nn.Dropout(dropout)
        self.layernorm2 = LayerNorm(embed_dim)

    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor]):
        attn_output, _ = self.attention(x, x, x, attn_mask=mask)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)
        
        return out2

class Transformer(nn.Module):
    def __init__(self, params: ModelArgs):
        super().__init__()
        self.embedding = nn.Embedding(params.input_dim, params.embed_dim)
        self.positional_encoding = PositionalEncoding(params.embed_dim, params.dropout)
        self.encoder_layers = nn.ModuleList([
            TransformerBlock(i, params.embed_dim, params.num_heads, params.ff_hidden_dim, params.dropout)
            for i in range(params.num_layers)
        ])
        self.decoder_layers = nn.ModuleList([
            TransformerBlock(i, params.embed_dim, params.num_heads, params.ff_hidden_dim, params.dropout)
            for i in range(params.num_layers)
        ])
        self.fc_out = nn.Linear(params.embed_dim, params.output_dim)

    def forward(self, input_tokens: torch.Tensor):
        input_embedded = self.embedding(input_tokens) * torch.sqrt(torch.tensor(self.params.embed_dim))
        input_encoded = self.positional_encoding(input_embedded)
        
        for encoder_layer in self.encoder_layers:
            input_encoded = encoder_layer(input_encoded, mask=None) 
        
        output_encoded = input_encoded 
        
        for decoder_layer in self.decoder_layers:
            output_encoded = decoder_layer(output_encoded, mask=None) 
        
        output = self.fc_out(output_encoded)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

model_args = ModelArgs(
    embed_dim=64,
    num_heads=8,
    ff_hidden_dim=128,
    num_layers=6,
    input_dim=10,
    output_dim=10,
    dropout=0.1
)

if __name__ == '__main__':
    transformer_block = TransformerBlock(
        layer_id=0,
        embed_dim=model_args.embed_dim,
        num_heads=model_args.num_heads,
        ff_hidden_dim=model_args.ff_hidden_dim,
        dropout=model_args.dropout
    )

    input_tensor = torch.rand(10, 2, model_args.embed_dim)  # (seq_len, batch_size, embed_dim)
    mask = None 

    output = transformer_block(input_tensor, mask)
    print("Output:", output) 
    print("Output shape:", output.shape) 

### 运行结果(**Q13**)：

![ALT](02py.png)

# Q13：尝试在你构建的transformer上训练数据，可以参考d2l中的做

    Beam Search是一种用于序列生成的启发式搜索算法，它在生成每个单词或符号时，都保留了一组（即“束”或"beam"）最有潜力的部分序列，而不是只选择当前概率最高的一个。这种方法可以有效地提高输出的质量，尤其是在机器翻译、文本摘要等任务中。

**Beam Search的基本步骤：**
初始化：开始时，将初始序列（通常是开始符号）加入到beam中。
迭代展开：对于beam中的每个序列，计算所有可能的下一个token的概率，并选择概率最高的num_beams个序列加入到新的beam中。
完成序列：当序列达到预定的长度或生成了结束符号时，将其从beam中移出，并作为候选的完成序列。
输出选择：在所有候选序列中选择最优的一个或多个序列作为输出。通常是基于序列的总体概率（如对数概率的和）来选择。
参数解释：
    
1. num_beams：
作用：定义了在每一步搜索中保留的序列数量。这个参数直接影响了搜索的空间大小。
值：一个正整数，例如num_beams=5表示在每一步都保留概率最高的5个序列。
2. top_k：
作用：在为每个序列生成下一个token时，只考虑概率最高的top_k个token。这可以避免考虑太多低概率的token，从而加速搜索过程。
值：一个正整数，例如top_k=50。
3. top_p（也称为核采样）：
作用：与top_k类似，但是它考虑的是概率质量而不是固定数量的token。具体来说，它只考虑累积概率达到top_p的token。
值：一个0到1之间的小数，例如top_p=0.9。
4. temperature：
作用：用于调节概率分布的平滑程度。较高的temperature值会使概率分布更加平滑，从而增加随机性和多样性；较低的temperature值会使得模型更加自信，输出更确定的预测。
值：一个大于0的数，通常接近1，例如temperature=1.0表示不改变分布，而temperature=0.5表示更倾向于高概率的token。


# Q15（选做）：下列代码节选自某框架，请根据上一题的学习完成代码补全....

In [ ]:
import torch
import torch.nn.functional as F

# 假设 logits 是模型生成的概率分布，其已经通过softmax处理
# logits.shape == (batch_size, seq_len, vocab_size)
logits = self.model.forward(tokens[:, prev_pos:cur_pos], prev_pos=cur_pos)

if temperature > 0:
    logits = logits / temperature
    probs = F.softmax(logits, dim=-1)
else:
    probs = logits

def sample_top_p(probs, p):
    """
    Perform top-p (nucleus) sampling on a probability distribution.

    Args:
        probs (torch.Tensor): Probability distribution tensor.
        p (float): Probability threshold for top-p sampling.

    Returns:
        torch.Tensor: Sampled token indices.

    Note:
        Top-p sampling selects the smallest set of tokens whose cumulative 
        probability mass exceeds the threshold p. The distribution is renormalized 
        based on the selected tokens.
    """
    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
    
    sorted_indices_to_remove = cumulative_probs > p
    if sorted_indices_to_remove.dim() > 1:
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

    sorted_probs[sorted_indices_to_remove] = 0
    sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)
    
    next_token = torch.multinomial(sorted_probs, num_samples=1)
    return sorted_indices.gather(-1, next_token)

temperatures = 1.0
top_p = 0.9

probs = F.softmax(logits[:, -1, :] / temperature, dim=-1)
next_tokens = sample_top_p(probs, top_p)

# Update tokens with next_tokens
tokens = torch.cat([tokens, next_tokens], dim=1)
